In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/연세대학교/YBIGTA/신입기수 프로젝트'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/연세대학교/YBIGTA/신입기수 프로젝트


## Call tomorrow's weather data

In [2]:
import urllib
import urllib.request
import json
from datetime import date, timedelta, datetime
from joblib import load
import pandas as pd
import numpy as np
import math

In [3]:
today = date.today()
formatted_today = today.strftime('%Y%m%d')
tomorrow = today + timedelta(days = 1)
formatted_tomorrow = tomorrow.strftime('%Y%m%d')

In [4]:
servicekey = 's+fQ9LDUrt9xJ9LSIp0R4+gJBR7eOiUpRHNKXMb6gaV844FL4oI+OYVOY+MC2Bff+Iq9bQWFeWrktswAfBtkyg=='
url= 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst'

queryParams = '?' + urllib.parse.urlencode(
    {
        urllib.parse.quote_plus('servicekey') : servicekey,
        urllib.parse.quote_plus('pageNo') : '1',
        urllib.parse.quote_plus('numOfRows') : '372',  # 12 * 31 = 372
       # 아님 12개 항목임
        urllib.parse.quote_plus('dataType') : 'JSON', # JSON, XML 두가지 포멧을 제공합니다.
        urllib.parse.quote_plus('base_date') : formatted_today, # 예보 받을 날짜를 입력합니다. 최근 1일간의 자료만 제공합니다.
        urllib.parse.quote_plus('base_time') : '1700', # 예보 시간을 입력합니다. 2시부터 시작하여 3시간 단위로 입력 가능합니다.
        urllib.parse.quote_plus('nx') : '48', # 울산 태양광 발전소 x 좌표입니다. '기상청18_동네예보 조회서비스_오픈API활용가이드.zip'에 포함 된 excel파일을 통해 확인 가능합니다.
        urllib.parse.quote_plus('ny') : '36' # 울산 태양광 발전소 y 좌표입니다. '기상청18_동네예보 조회서비스_오픈API활용가이드.zip'에 포함 된 excel파일을 통해 확인 가능합니다.
    }
)

response = urllib.request.urlopen(url + queryParams).read()
response = json.loads(response)

In [5]:
fcst_df = pd.DataFrame(columns=['Forecast_date', 'Forecast_hour', 'WindDirection', 'WindSpeed', 'Cloud', 'Rainfall', 'Humidity', 'Temperature'])

row_idx = 0  # row_idx 초기화

for data in response['response']['body']['items']['item']:
    fcst_df.loc[row_idx, 'Forecast_date'] = data['fcstDate']
    fcst_df.loc[row_idx, 'Forecast_hour'] = data['fcstTime']

    if data['category'] == 'REH':
        fcst_df.loc[row_idx, 'Humidity'] = float(data.get('fcstValue', 'NaN'))
    elif data['category'] == 'PCP':
        fcst_df.loc[row_idx, 'Rainfall'] = str(data.get('fcstValue', 'NaN'))
    elif data['category'] == 'TMP':
        fcst_df.loc[row_idx, 'Temperature'] = float(data.get('fcstValue', 'NaN'))
    elif data['category'] == 'SKY':
        fcst_df.loc[row_idx, 'Cloud'] = float(data.get('fcstValue', 'NaN'))
    elif data['category'] == 'VEC':
        fcst_df.loc[row_idx, 'WindDirection'] = float(data.get('fcstValue', 'NaN'))
    elif data['category'] == 'WSD':
        fcst_df.loc[row_idx, 'WindSpeed'] = float(data.get('fcstValue', 'NaN'))
        row_idx += 1  # 다음 행으로 이동

In [6]:
# Forecast_date와 Forecast_hour 열을 datetime 형식으로 변환
fcst_df['Forecast_date'] = pd.to_datetime(fcst_df['Forecast_date'], format='%Y%m%d')
fcst_df['Forecast_hour'] = fcst_df['Forecast_hour'].astype(str).str.zfill(4)  # 시간을 4자리 문자열로 변환
fcst_df['Forecast_hour'] = fcst_df['Forecast_hour'].str[:2].astype(int)  # 앞의 2자리를 추출하여 정수로 변환

In [7]:
fcst_df = fcst_df.rename(columns={'Forecast_date': 'date', 'Forecast_hour': 'hour'})

# date가 '
# 내일 날짜인 데이터만 추출하여 tomorrow_df에 저장
tomorrow_df = fcst_df[(fcst_df['date'] == formatted_tomorrow)].copy()

In [8]:
tomorrow_df.replace('강수없음', int('0'), inplace=True)

In [9]:
# Rainfall 열 값 변경
rainfall_mapping = {'0': 0, '1mm 미만': 1, '1.0mm': 1, '2.0mm': 1, '3.0mm': 1}
tomorrow_df['Rainfall'] = tomorrow_df['Rainfall'].astype(str).map(rainfall_mapping).fillna(2).astype(int)

# Cloud 값 보정
cloud_mapping = {1: 2, 2: 4, 3: 8, 4: 10}
tomorrow_df['Cloud'] = tomorrow_df['Cloud'].map(cloud_mapping)

# hour 값 0~23 을 1~24로 수정
tomorrow_df['hour'] = tomorrow_df['hour'] + 1

In [10]:
an = [1.000110,0.034221,0.000719]
bn = [0,0.001280,0.000077]
cn = [0.006918,-0.399912,-0.006758,-0.002697]
dn = [0,0.070257,0.000907,0.000148]


S = 1367 #solar constant
L = 33.3 #latitude
L_rad = np.deg2rad(L) #latitude를 rad으로 변환

tomorrow_df['date'] = pd.to_datetime(tomorrow_df['date'])
d = tomorrow_df['date'].dt.dayofyear
t = 2*np.pi*d/365

In [11]:
LN = datetime(2024, 2, 9, 12, 0, 0) # local noon time

In [12]:
# sun-earth distance  : r0 값을 알 수 없어서, a로 근사한 식을 이용함
r0_r2 = np.zeros(t.shape)
for i in range(0,3) :
    r0_r2 = r0_r2+an[i]*np.cos(i*t)+bn[i]*np.sin(i*t)

# declination angle
delta_rad = np.zeros(t.shape)
for i in range(0,4) :
    delta_rad = delta_rad+cn[i]*np.cos(i*t)+dn[i]*np.sin(i*t)

In [13]:
# Solar insolation for hour gap
# Q12는 태양 남중 12시로 가정한거고, Q13은 남중 13시로 가정한검니당
Q = np.zeros(t.shape)
gap = np.pi/12
tomorrow_df['Q12'] = S*r0_r2*((np.sin(L_rad)*np.sin(delta_rad))+((24/np.pi)*np.cos(L_rad)*np.cos(delta_rad)*np.sin(np.pi/24)*np.cos((tomorrow_df.hour-12)*gap)))
tomorrow_df['Q12'] = tomorrow_df['Q12'].apply(lambda x: max(0, x))
tomorrow_df['Q13'] = S*r0_r2*((np.sin(L_rad)*np.sin(delta_rad))+((24/np.pi)*np.cos(L_rad)*np.cos(delta_rad)*np.sin(np.pi/24)*np.cos((tomorrow_df.hour-13)*gap)))
tomorrow_df['Q13'] = tomorrow_df['Q13'].apply(lambda x: max(0, x))
tomorrow_df['Q_mean'] = (tomorrow_df['Q12']+tomorrow_df['Q13'])/2

In [14]:
tomorrow_df

,date,hour,WindDirection,WindSpeed,Cloud,Rainfall,Humidity,Temperature,Q12,Q13,Q_mean
6,2024-02-17,1,172.0,2.1,2,0,80.0,6.0,0.000000,0.000000,0.000000
7,2024-02-17,2,160.0,2.0,2,0,80.0,6.0,0.000000,0.000000,0.000000
8,2024-02-17,3,151.0,2.1,8,0,80.0,6.0,0.000000,0.000000,0.000000
9,2024-02-17,4,144.0,2.4,2,0,80.0,6.0,0.000000,0.000000,0.000000
10,2024-02-17,5,143.0,2.6,2,0,80.0,5.0,0.000000,0.000000,0.000000
11,2024-02-17,6,146.0,2.7,2,0,80.0,5.0,0.000000,0.000000,0.000000
12,2024-02-17,7,151.0,2.9,2,0,80.0,6.0,136.177136,0.000000,68.088568
13,2024-02-17,8,152.0,2.7,2,0,85.0,5.0,411.508808,136.177136,273.842972
14,2024-02-17,9,147.0,2.7,2,0,85.0,6.0,647.941474,411.508808,529.725141
15,2024-02-17,10,149.0,2.6,2,0,75.0,10.0,829.362640,647.941474,738.652057


## Preprocess dataset

In [15]:
tomorrow_df = tomorrow_df.dropna()
tomorrow_df.reset_index(drop = True, inplace = True)

In [16]:
tomorrow_df['WindDirection_cos'] = tomorrow_df['WindDirection'].apply(lambda x : math.cos(x))
tomorrow_df['WindDirection_sin'] = tomorrow_df['WindDirection'].apply(lambda x : math.sin(x))

In [17]:
tomorrow_df['Year'] = tomorrow_df['date'].apply(lambda x : x.year)
tomorrow_df['Month'] = tomorrow_df['date'].apply(lambda x : x.month)
tomorrow_df['Day'] = tomorrow_df['date'].apply(lambda x : x.day)

In [18]:
tomorrow_df = tomorrow_df.drop(columns = ['WindDirection', 'Q12', 'Q13', 'date'])

In [19]:
columns_to_scale = ['WindSpeed', 'Cloud', 'Rainfall', 'Humidity', 'Temperature', 'Q_mean']
sd_scaler = load('sd_scaler.bin')

tomorrow_df[columns_to_scale] = sd_scaler.transform(tomorrow_df[columns_to_scale])

In [20]:
PCA_df = tomorrow_df.drop(columns = ['WindSpeed', 'WindDirection_cos', 'WindDirection_sin'])

pca = load('pca_model.joblib')
PCA_df = pca.transform(PCA_df)
principalDf = pd.DataFrame(data = PCA_df, columns = ['PC1', 'PC2', 'PC3'])

tomorrow_df = pd.concat([tomorrow_df[['WindSpeed', 'WindDirection_cos', 'WindDirection_sin']], principalDf], axis = 1)

## Predict Wind Power

In [22]:
import torch
import torch.nn as nn

In [23]:
class Windpower_MLP(nn.Module):
    def __init__(self):
        super(Windpower_MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(6, 32), nn.ReLU(), # nn.Dropout(),
            nn.Linear(32, 64), nn.ReLU(),
            nn.Linear(64, 10), nn.ReLU(), # nn.Dropout(),
            nn.Linear(10, 1)
        )

    def forward(self, x):
        return self.layers(x)

In [24]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(DEVICE)

cpu


In [28]:
inputs = tomorrow_df.to_numpy(dtype = np.float32)
inputs = torch.tensor(inputs)

In [29]:
model = torch.load('windpower_model.pt')
outputs = model(inputs)

print(outputs)

tensor([[334.9251],
        [353.8169],
        [216.7489],
        [205.5541],
        [243.5869],
        [265.8662],
        [217.6865],
        [257.8024],
        [334.2410],
        [285.1354],
        [278.2344],
        [263.2161],
        [245.3880],
        [275.1082],
        [219.0217],
        [206.2142],
        [250.5663],
        [224.1606],
        [212.7172],
        [202.2499],
        [165.9212],
        [227.9037],
        [199.7236],
        [176.3301]], grad_fn=<AddmmBackward0>)
